# Мегалайн

Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением accuracy. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте accuracy на тестовой выборке самостоятельно.

1. Откройте файл с данными и изучите его.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import random

df = pd.read_csv('users_behavior.csv')

df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. Известно:
- сalls — количество звонков,
- minutes — суммарная длительность звонков в минутах,
- messages — количество sms-сообщений,
- mb_used — израсходованный интернет-трафик в Мб,
- is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).


2. Разделите исходные данные на обучающую, валидационную и тестовую выборки.

Спрятанной тестовой выборки нет. Значит, данные нужно разбить на три части: обучающую, валидационную и тестовую. Размеры тестового и валидационного наборов обычно равны. Исходные данные разбивают в соотношении 3:1:1.

In [2]:
#Сначала разбили данные на 2 выборки, первая в которой - обучающая
df_train, df_valid_test = train_test_split(df,test_size=.4, random_state=12345)
#Теперь разбиваем вторую выборку на валидационную и тестовую пополам
df_valid, df_test       = train_test_split(df_valid_test, test_size=.5, random_state=12345)

3. Исследуйте качество разных моделей, меняя гиперпараметры. Кратко напишите выводы исследования.

Займемся решающим деревом.

In [3]:
#Разделяем признаки для обучающей выборки
features_train = df_train.drop(['is_ultra'],axis=1)
target_train   = df_train['is_ultra']
#Разделяем признаки для валидационной выборки
features_valid = df_valid.drop(['is_ultra'],axis=1)
target_valid   = df_valid['is_ultra']

#Будем варьировать гиперпараметр - глубину дерева
for m_d in range(2,21,2):
    decision_tree_model = DecisionTreeClassifier(random_state=12345, max_depth=m_d)
    decision_tree_model.fit(features_train,target_train)
    predictions         = decision_tree_model.predict(features_valid)
    accuracy            = round(accuracy_score(target_valid,predictions),4)
    predictions_        = decision_tree_model.predict(features_train)
    accuracy_           = round(accuracy_score(target_train,predictions_),4)
    print('Глубина дерева = ' + str(m_d) + '. Accuracy_train = ' + str(accuracy_) + '. Accuracy_valid = ' + str(accuracy))
    print()

Глубина дерева = 2. Accuracy_train = 0.7879. Accuracy_valid = 0.7823

Глубина дерева = 4. Accuracy_train = 0.8107. Accuracy_valid = 0.7792

Глубина дерева = 6. Accuracy_train = 0.8377. Accuracy_valid = 0.7838

Глубина дерева = 8. Accuracy_train = 0.8626. Accuracy_valid = 0.7792

Глубина дерева = 10. Accuracy_train = 0.889. Accuracy_valid = 0.7745

Глубина дерева = 12. Accuracy_train = 0.9253. Accuracy_valid = 0.7621

Глубина дерева = 14. Accuracy_train = 0.9554. Accuracy_valid = 0.7589

Глубина дерева = 16. Accuracy_train = 0.9787. Accuracy_valid = 0.7341

Глубина дерева = 18. Accuracy_train = 0.9886. Accuracy_valid = 0.7309

Глубина дерева = 20. Accuracy_train = 0.9933. Accuracy_valid = 0.7216



В случае с моделью решающего дерева, мы изменяли гиперпараметр (глубину дерева), чтобы добиться максимального качества, которое определяем через accuracy (отношение количества правильных ответов к количеству ответов). С увеличением данного гиперпараметра качество модели на обучающей выборке, как ни странно, растет. Однако, нельзя переусердствовать с этим значением, посколько наша модель может стать переобученной (она зазубрила все ответы и зависимости между переменными не поняла). Но и слишом низким данный гиперпараметр быть не должен, инче модель будет недообученной.

В цикле выше мы проверяли нашу модель по валидационной и тестовой выборке для значений глубины дерева от 2 до 20 с шагом в 2. Начиная со значения 10 accuracy начинает уверенно падать, поэтому остановимся на данном значении.

Теперь займемся случайным лесом. Будем менять количество деревьев.

Алгоритм обучает большое количество независимых друг от друга деревьев, а потом принимает решение на основе голосования. Случайный лес помогает улучшить результат предсказания и избежать переобучения.

In [4]:
for n_e in range(10,51,10):
    random_tree_model = RandomForestClassifier(random_state=12345,n_estimators=n_e,max_depth=8)
    random_tree_model.fit(features_train,target_train)
    predictions       = random_tree_model.predict(features_valid)
    accuracy          = round(accuracy_score(target_valid,predictions),4)
    predictions_      = random_tree_model.predict(features_train)
    accuracy_         = round(accuracy_score(target_train,predictions_),4)
    print('Количество деревьев = ' + str(n_e) + '. Accuracy_train = ' + str(accuracy_) + '. Accuracy_valid = ' + str(accuracy))
    print()

Количество деревьев = 10. Accuracy_train = 0.8698. Accuracy_valid = 0.7963

Количество деревьев = 20. Accuracy_train = 0.8714. Accuracy_valid = 0.7978

Количество деревьев = 30. Accuracy_train = 0.8729. Accuracy_valid = 0.7994

Количество деревьев = 40. Accuracy_train = 0.875. Accuracy_valid = 0.8087

Количество деревьев = 50. Accuracy_train = 0.8755. Accuracy_valid = 0.8072



При варьировании гиперпараметра модели случайного леса (n_estimators) мы шли с шагом 10 от 10 до 50. Выберем модель со значением 40, так как при нем accuracy получается наибольшей для валидационной выборки и дальше особо не меняется. Здесь же мы меняли глубину вручную и получили наилучшие результаты для 8.

В заключение попробуем модель логистической регрессии. 

В логистической регрессии параметров мало. Что-либо вызубрить по признакам в формуле не выйдет, поэтому и вероятность переобучения невелика.

In [5]:
logistic_regression_model = LogisticRegression(random_state=12345,solver='lbfgs')
logistic_regression_model.fit(features_train,target_train)

predictions = logistic_regression_model.predict(features_valid)
accuracy    = round(accuracy_score(target_valid,predictions),4)
print(accuracy)

0.7107


У модели логистической регрессии нет гиперпараметров, поэтому и менять тут нечего. Получили значение accuracy 0.75 и теперь возьмем лучшие варианты моделей решающего дерева и случайного леса и проверим по ним тестовую выборку, предварительно объединив обучающую выборку с валидационной.

4. Проверьте качество модели на тестовой выборке.

In [6]:
#Разделяем признаки для тестовой выборки
features_test = df_test.drop(['is_ultra'],axis=1)
target_test   = df_test['is_ultra']

#Объединим обучающую и валидационную выборки, чтобы воспользоваться суммой при обучении
df_new_train       = pd.concat([df_train,df_valid])
features_new_train = df_new_train.drop(['is_ultra'],axis=1)
target_new_train   = df_new_train['is_ultra']

#Лучшее решающее дерево
decision_tree_model       = DecisionTreeClassifier(random_state=12345, max_depth=10)
decision_tree_model.fit(features_new_train,target_new_train)
decision_tree_predictions = decision_tree_model.predict(features_test)
decision_tree_accuracy    = round(accuracy_score(target_test,decision_tree_predictions),4)

#Лучший случайный лес
random_tree_model       = RandomForestClassifier(random_state=12345,n_estimators=40,max_depth=8)
random_tree_model.fit(features_new_train,target_new_train)
random_tree_predictions = random_tree_model.predict(features_test)
radom_tree_accuracy     = round(accuracy_score(target_test,random_tree_predictions),4)

#Логистическая регрессия
logistic_regression_model       = LogisticRegression(random_state=12345,solver='lbfgs')
logistic_regression_model.fit(features_new_train,target_new_train)
logistic_regression_predictions = logistic_regression_model.predict(features_test)
logistic_regression_accuracy    = round(accuracy_score(target_test,logistic_regression_predictions),4)

print('Accuracy для решающего дерева        = ' + str(decision_tree_accuracy))
print('Accuracy для случайного леса         = ' + str(radom_tree_accuracy))
print('Accuracy для логистической регрессии = ' + str(logistic_regression_accuracy))

Accuracy для решающего дерева        = 0.7932
Accuracy для случайного леса         = 0.7994
Accuracy для логистической регрессии = 0.6843


В результате определения наилучших гиперпараметров для решающего дерева и случайного леса, мы получили, что обе эти модели дают на тестовой выборке accuracy более 0.75. Случайный лес с крохотным отставанием становится на первое место по качеству.

5. Дополнительное задание: проверьте модели на вменяемость.

Дла задачи классификации проверку на вменяемость будем делать по случайной модели.

In [7]:
#Сгенерируем случайные предсказания и закинем их в наш датасет
random_predictions = []
for i in range(len(df.index)):
    random_predictions.append(random.randint(0,1))
random_predictions = pd.Series(random_predictions,index=df.index)  
df['prediction'] = random_predictions

In [8]:
def sanity_check(row):
    if row['is_ultra'] == row['prediction']:
        return 1
    else:
        return 0

In [9]:
df['sanity'] = df.apply(sanity_check,axis=1)
num_of_right_answers = df['sanity'].sum()
accuracy = round(num_of_right_answers / df.shape[0],4)
print('Accuracy случайной модели = ' + str(accuracy))

Accuracy случайной модели = 0.5081


Получили то, что и ожидали - случайная модель предсказывает целевой параметр правильно с вероятностью 50%. Все три модели сработали лучше случайной и это означает, что они вменяемы.